In [ ]:
pip install sentence_transformers pandas numpy datasets accelerate tqdm bitsandbytes huggingface_hub hf_transfer

  Using cached sentence_transformers-5.1.2-py3-none-any.whl.metadata (16 kB)
  Using cached pandas-2.3.3-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
  Using cached datasets-4.4.1-py3-none-any.whl.metadata (19 kB)
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
  Using cached huggingface_hub-1.1.5-py3-none-any.whl.metadata (13 kB)
  Using cached hf_transfer-0.1.9-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.7 kB)
  Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached scipy-1.16.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (62 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached regex-2025.11.3-cp312-cp312-manylinux2014_x86_64.manylinux_

In [ ]:
# =========================================
# 0. 라이브러리 & 기본 설정
# =========================================
import torch
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import CrossEncoder
from datasets import load_dataset
from transformers import AutoTokenizer
from tqdm import tqdm
import numpy as np
import random

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


# =========================================
# 1. 데이터 불러오기 (query, doc)
# =========================================
dataset = load_dataset("HJUNN/Art_Therapy_caption_train_dataset")["train"]
train_df = dataset.to_pandas()

positive_pairs = list(zip(train_df["query"], train_df["doc"]))
print("Positive pairs:", len(positive_pairs))


# =========================================
# 2. Hard Negative Sampling (HTP 최적화 버전)
# =========================================
from sentence_transformers import SentenceTransformer, util

embed_model = SentenceTransformer("HJUNN/bge-m3b-Art-Therapy-embedding-fine-tuning")

# 모든 문서 임베딩
all_docs = list(dict.fromkeys(train_df["doc"].tolist()))
doc_emb = embed_model.encode(all_docs, convert_to_tensor=True, normalize_embeddings=True)

hard_negatives = []

for q, pos_doc in tqdm(positive_pairs, desc="Hard Negative Mining"):
    q_emb = embed_model.encode(q, convert_to_tensor=True, normalize_embeddings=True)
    scores = util.cos_sim(q_emb, doc_emb)[0]

    # 너무 유사한 문서 제거 (양성문서와 착각)
    mask1 = scores < 0.60
    # 너무 먼 문서 제거 (학습 영향 없음)
    mask2 = scores > 0.30

    filtered_idx = torch.where(mask1 & mask2)[0]

    if len(filtered_idx) == 0:
        continue

    # 랜덤으로 2개만 선택
    selected = filtered_idx[torch.randperm(len(filtered_idx))[:2]]

    for idx in selected:
        neg_doc = all_docs[idx]
        hard_negatives.append((q, pos_doc, neg_doc))

print("Hard negative triples:", len(hard_negatives))



# =========================================
# 3. Dataset 정의
# =========================================
class TripleDataset(Dataset):
    def __init__(self, triples):
        self.triples = triples

    def __len__(self):
        return len(self.triples)

    def __getitem__(self, idx):
        return self.triples[idx]  # q, pos, neg


dataset = TripleDataset(hard_negatives)
loader = DataLoader(dataset, batch_size=4, shuffle=True)



# =========================================
# 4. CrossEncoder 로드
# =========================================
model_name = "BAAI/bge-reranker-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

cross_encoder = CrossEncoder(
    model_name,
    num_labels=1,
    device=device
)

optimizer = torch.optim.AdamW(cross_encoder.model.parameters(), lr=2e-5)

# BCE Loss + Label smoothing
loss_fn = torch.nn.BCEWithLogitsLoss()


# =========================================
# 5. 파인튜닝 (BCE 방식, 매우 안정적)
# =========================================
epochs = 5

for epoch in range(epochs):
    total_loss = 0
    cross_encoder.model.train()

    for q, pos, neg in tqdm(loader, desc=f"Epoch {epoch+1}/{epochs}"):

        batch_size = len(q)

        # -----------------------------
        # Positive pair
        # -----------------------------
        pos_inputs = tokenizer(
            list(q),
            list(pos),
            padding=True,
            truncation=True,
            max_length=256,
            return_tensors="pt"
        ).to(device)

        pos_logits = cross_encoder.model(**pos_inputs).logits.squeeze()
        pos_labels = torch.ones(batch_size).to(device) * 0.9   # label smoothing

        # -----------------------------
        # Negative pair
        # -----------------------------
        neg_inputs = tokenizer(
            list(q),
            list(neg),
            padding=True,
            truncation=True,
            max_length=256,
            return_tensors="pt"
        ).to(device)

        neg_logits = cross_encoder.model(**neg_inputs).logits.squeeze()
        neg_labels = torch.zeros(batch_size).to(device) + 0.1  # label smoothing

        # -----------------------------
        # Loss 계산
        # -----------------------------
        logits = torch.cat([pos_logits, neg_logits])
        labels = torch.cat([pos_labels, neg_labels])

        loss = loss_fn(logits, labels)

        # -----------------------------
        # 역전파
        # -----------------------------
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(cross_encoder.model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()

    print(f"[Epoch {epoch+1}] Loss: {total_loss/len(loader):.4f}")

print("🔥 Training Finished!")



# =========================================
# 6. 모델 저장
# =========================================
save_path = "./htp_cross_encoder_bce"
cross_encoder.save(save_path)
print("모델 저장 완료:", save_path)



# =========================================
# 7. 평가 함수
# =========================================
def evaluate_scores(model, query, candidates):
    inputs = [(query, c) for c in candidates]
    logits = model.predict(inputs, convert_to_numpy=True)

    # softmax 정규화
    logits = torch.tensor(logits)
    probs = torch.softmax(logits, dim=0).tolist()

    ranked = sorted(zip(candidates, probs), key=lambda x: x[1], reverse=True)
    return ranked



# =========================================
# 8. 검증 샘플 테스트
# =========================================
valid_dataset = load_dataset("HJUNN/Art_Therapy_caption_valid_dataset")["validation"]
valid_df = valid_dataset.to_pandas()

test_queries = [
    valid_df["query"][0],
    valid_df["query"][55],
    valid_df["query"][85]
]

candidates = [
    valid_df["doc"][i] for i in [0,7,20,55,60,65,85,90,95]
]

print("\n==============================")
print("🔥 학습 후 CrossEncoder 결과")
print("==============================\n")

for q in test_queries:
    print("[Query]", q)
    ranked = evaluate_scores(cross_encoder, q, candidates)
    for rank, (doc, score) in enumerate(ranked, 1):
        print(f"{rank}위 | 점수 {score:.4f} | 문서: {doc[:50]}...")
    print("-----------------------------------------")
ㅠ

Using device: cuda
Positive pairs: 573


Hard Negative Mining: 100%|██████████| 573/573 [00:05<00:00, 102.17it/s]


Hard negative triples: 1143


Epoch 1/5: 100%|██████████| 286/286 [00:27<00:00, 10.29it/s]


[Epoch 1] Loss: 0.4875


...

Epoch 4/5: 100%|██████████| 286/286 [00:27<00:00, 10.37it/s]


[Epoch 4] Loss: 0.3583


Epoch 5/5: 100%|██████████| 286/286 [00:27<00:00, 10.43it/s]


[Epoch 5] Loss: 0.3525
🔥 Training Finished!
모델 저장 완료: ./htp_cross_encoder_bce

🔥 학습 후 CrossEncoder 결과

[Query] 그림의 집은 넓고 크게 그려져 있으며, 중앙에 자리 잡고 있습니다.
1위 | 점수 0.2124 | 문서: 1.<집 그림의 분석결과 정리>


 집 크기 
상태: 크게 그리는 경우 
가능한 분석: ...
2위 | 점수 0.1015 | 문서: 3. 항목: 집의 형태 
상태: 양옥집 
가능한 분석: 양옥집을 그리는 피험자의 경우 타인...
3위 | 점수 0.0985 | 문서: 9. 두상비율
- 적당: 피험자가 성취동기가 높을수록 두상비율을 균형 있게 그리는 경향이 ...
4위 | 점수 0.0980 | 문서: 11. 다리그림
- 짧다: 피험자가 현실적일수록 다리를 짧게 그리는 경향이 있다.


다리...
5위 | 점수 0.0979 | 문서: 7. 목그림
- 생략: 피험자가 우울할수록, 반사회적 성향이 강할수록, 또는 인내심이 강할...
6위 | 점수 0.0979 | 문서: 5. 동물유무 
- 있다: 피험자가 자신감이 클수록 동물을 그리는 경향이 있다.
 
뿌리위...
7위 | 점수 0.0979 | 문서: 7. 가지모양 
 - (가지모양 이미지) 피험자가 창의적일수록, 공감을 잘할수록 위로 뾰족...
8위 | 점수 0.0979 | 문서: 7. 항목: 창문 크기 
상태: 크게 
가능한 분석: 피험자가 호기심이 많을수록, 야심이 ...
9위 | 점수 0.0979 | 문서: 8. 줄기에서 뻗은 새끼가지 
있다: 피험자가 야심 있고 목표지향적일수록 줄기에서 뻗은 새...
-----------------------------------------
[Query] 나무 뿌리는 땅 속 깊이 자리 잡고 있으며, 여백에는 잔디가 고르게 깔려 있습니다. 동물이 나무 옆에 서 있는 모습도 보입니다.
1위 | 점수 0.2136 | 문서: 5. 동물유무 
- 있다: 피험자가 자신감이 클수록 동물을 

NameError: name 'ᅲ' is not defined